In [1]:
from riken.rnn.rnn_keras_with_psiblast import *
from sklearn.metrics import roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
from riken.viz_utils import visualize_attention, to_2d_array, visualize_attention_2d
from riken.protein_io.prot_features import idx_to_char
from keras.models import Model
from riken.word2vec.classification_tools import roc_score

from tqdm import tqdm


RANDOM_STATE = 42
MAXLEN = 500
DATA_PATH = "/home/pierre/riken/data/riken_data/complete_from_xlsx_v2COMPLETE.tsv"
KEY_TO_PREDICT = "is_allergenic"
LOG_DIR = "/home/pierre/riken/riken/rnn/logs/riken_data_psiblast/logs_riken_data_psiblast_1_transfer"
TRANSFER_PATH = None
SPLITTER = data_op.group_shuffle_indices
PSSM_FORMAT_FILE = "/home/pierre/riken/data/psiblast/riken_data_v2/{}_pssm.txt"

Using TensorFlow backend.


In [2]:
df = pd.read_csv(DATA_PATH, sep='\t', index_col=0).dropna()
df = df.loc[df.seq_len >= 50, :]

sequences, y = df['sequences'].values, df[KEY_TO_PREDICT]
y = pd.get_dummies(y).values
X = pad_sequences([[prot_features.safe_char_to_idx(char) for char in sequence]
                   for sequence in sequences], maxlen=MAXLEN)
indices = df.index.values

# Get train test indices before getting all features (potentially less
# FOR CONSISTENCY PURPOSES

# groups = None if GROUPS is None else df[GROUPS].values
# train_inds, test_inds = SPLITTER(sequences, y, groups)

train_inds, test_inds = np.where(df.is_train)[0], np.where(df.is_train==False)[0]

X, pssm, y = get_all_features(X, y, indices, pssm_format_fi=PSSM_FORMAT_FILE)

def convert_indices(old_to_new_dic, indices):
    res_ind = []
    for ind in indices:
        if ind in old_to_new_dic:
            res_ind.append(old_to_new_dic[ind])
    return np.array(res_ind)

print(np.intersect1d(train_inds, test_inds))
assert len(np.intersect1d(train_inds, test_inds)) == 0
print(train_inds.shape, test_inds.shape)
Xtrain, Xtest, ytrain, ytest = X[train_inds], X[test_inds], y[train_inds], y[test_inds]
pssm_train, pssm_test = pssm[train_inds], pssm[test_inds]

print(pssm_train[0])
print(pssm_test[0])

  0%|          | 0/12285 [00:00<?, ?it/s]/home/pierre/riken/riken/rnn/rnn_keras_with_psiblast.py:41: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  pssm = pd.read_csv(pssm_path, sep=' ', skiprows=2, skipfooter=6, skipinitialspace=True)\
  3%|▎         | 365/12285 [00:08<04:28, 44.36it/s]

[Errno 2] No such file or directory: '/home/pierre/riken/data/psiblast/riken_data_v2/baa04558.1_pssm.txt'
Error!


  4%|▍         | 538/12285 [00:12<04:36, 42.50it/s]

[Errno 2] No such file or directory: '/home/pierre/riken/data/psiblast/riken_data_v2/caa09885.1_pssm.txt'
Error!


  5%|▍         | 555/12285 [00:13<04:38, 42.07it/s]

...

[Errno 2] No such file or directory: '/home/pierre/riken/data/psiblast/riken_data_v2/akj77990.1_pssm.txt'
Error!
[Errno 2] No such file or directory: '/home/pierre/riken/data/psiblast/riken_data_v2/bat62430.1_pssm.txt'
Error!


 15%|█▌        | 1894/12285 [00:36<03:20, 51.86it/s]

[Errno 2] No such file or directory: '/home/pierre/riken/data/psiblast/riken_data_v2/4zce_a_pssm.txt'
Error!


100%|██████████| 12285/12285 [04:14<00:00, 48.23it/s]


(12285, 500)
(12285, 2)
(12285, 500, 42)
12285 examples
[]
(11061,) (1224,)
[[ 0.    0.    0.   ...  0.    0.    0.  ]
 [ 0.    0.    0.   ...  0.    0.    0.  ]
 [ 0.    0.    0.   ...  0.    0.    0.  ]
 ...
 [ 3.   -4.   -4.   ...  0.    1.62  0.36]
 [-2.   -1.   -2.   ... 15.    0.33  0.16]
 [-3.   -4.   -5.   ... 40.    0.85  0.28]]
[[ 0.    0.    0.   ...  0.    0.    0.  ]
 [ 0.    0.    0.   ...  0.    0.    0.  ]
 [ 0.    0.    0.   ...  0.    0.    0.  ]
 ...
 [-1.   -2.    2.   ...  0.    0.55  0.14]
 [-2.   -1.    1.   ...  0.    0.98  0.23]
 [-2.   -2.   -3.   ...  0.    1.9   0.2 ]]


In [3]:
train_protein_names, test_protein_names = indices[train_inds], indices[test_inds]

In [4]:
from keras.models import load_model

mdl = load_model('logs/riken_data_psiblast/logs_riken_data_psiblast_1_NO_transfer/weights.12-0.37.hdf5')
ypred = mdl.predict([Xtest, pssm_test])

test_df = df.iloc[test_inds]
test_df.loc[:, 'y_pred'] = ypred[:, 1]

/home/pierre/.conda/envs/python3/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/pierre/.conda/envs/python3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [11]:
roc_auc_score(test_df.is_allergenic, test_df.y_pred)
roc_score(ytest[:, 1], ypred[:, 1])

0.9312413856202205

In [6]:
attention = mdl.get_layer(name='activation_1').output
attention_model = Model(inputs=mdl.inputs, outputs=attention)

# Save all attention viz

In [8]:
import os
def save_attentions(attentions, characters, y_true, y_pred, protein_names, dire):
    for indice, name in enumerate(tqdm(protein_names)):
        seq = characters[indice]
        first_non_null = [ind for ind in range(len(seq)) if seq[ind]!=''][0]
        seq = seq[first_non_null:]
        att = atts[indice][first_non_null:]
        assert(len(att)==len(seq))
        fig = plt.figure()
        ax = visualize_attention_2d(seq, att, ncols=20)
        fig.suptitle('protein name: {name}; allergen: {is_allergen}; predicted score: {score}'
                .format(name=name, is_allergen=y_true[indice], score=y_pred[indice]))
        fig.savefig(os.path.join(dire, '{}_attention.png'.format(name)))
        plt.close()
        

In [10]:
seqs, pssms = Xtest, pssm_test
atts = attention_model.predict([seqs, pssms])
seqs_str = [[idx_to_char[idce] for idce in seq] for seq in seqs]
ypred = mdl.predict([seqs, pssms])[:, 1]
ytrue = ytest[:, 1]

save_attentions(atts, seqs_str, ytrue, ypred, test_protein_names, 'tra_attentions')

100%|██████████| 1224/1224 [07:09<00:00,  2.85it/s]


In [11]:
!mkdir train_attention

# Experiments

In [14]:
def roc_auc(df):
    return roc_auc_score(df.is_allergenic, df.y_pred)

is_long = test_df.seq_len >= 500
test_df.groupby(is_long).apply(roc_auc)

seq_len
False    0.936191
True     0.897737
dtype: float64

In [14]:
is_long.sum()

193

In [15]:
test_df[is_long].seq_len.describe()

count     193.000000
mean      719.119171
std       345.191238
min       500.000000
25%       527.000000
50%       608.000000
75%       796.000000
max      3099.000000
Name: seq_len, dtype: float64